# Two-layer convolutional neural network

In [2]:
import sys
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras import callbacks
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l2
import time
import datetime

## Parameters

In [3]:
epochs = 20

In [4]:
train_data_path = '../subdatasets/Train'
validation_data_path = '../subdatasets/Test'

In [5]:
"""
Parameters
"""
img_width, img_height = 224, 224
batch_size = 32
samples_per_epoch = 1000
test_steps = 210

nb_filters1 = 32
nb_filters2 = 64
conv1_size = 3
conv2_size = 2

pool_size = 2
classes_num = 2
lr2= 0.001

## Model design

In [6]:
model = Sequential()
model.add(Convolution2D(nb_filters1, (conv1_size, conv1_size), padding ="same", input_shape=(img_width, img_height, 3),kernel_regularizer=l2(lr2)))
model.add(Dropout(0.3))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))


model.add(Convolution2D(nb_filters2, (conv2_size, conv2_size), padding ="same" ,kernel_regularizer=l2(lr2)))
model.add(Activation("relu"))
model.add(Dropout(0.3))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Flatten())
model.add(Dense(256,kernel_regularizer=l2(lr2)))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(classes_num, activation='softmax',kernel_regularizer=l2(lr2)))

model.compile(loss='sparse_categorical_crossentropy',
              #optimizer=optimizers.RMSprop(lr=lr),
              optimizer=optimizers.Adam(),
              metrics=['accuracy'])


In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 32)      896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 224, 224, 32)      0         
_________________________________________________________________
activation_1 (Activation)    (None, 224, 224, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 64)      8256      
_________________________________________________________________
activation_2 (Activation)    (None, 112, 112, 64)      0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 112, 112, 64)      0         
__________

## Save model

In [8]:
target_dir = '../models/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
model.save('../models/model-simple.h5')


## Data input for training

### Training Data 

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=10,
    zoom_range=0.2,
    horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

Found 484 images belonging to 2 classes.


### Test Data

In [10]:
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    validation_data_path,
    batch_size=batch_size,
    shuffle=False,
    target_size=(img_height, img_width),
    class_mode='binary')

Found 210 images belonging to 2 classes.


## Train Model. Saving weights as test evaluation increases

In [11]:
"""
Tensorboard log
"""
log_dir = './tf-log/ModelSimple'+"_"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
tb_cb = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0,write_graph=True, write_images=True)

In [ ]:
# checkpoint
filepath="../models/weights-simple-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint,tb_cb]

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=samples_per_epoch // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_steps // batch_size,
    callbacks=callbacks_list, verbose=1)

Epoch 1/20
